> For more information how to run the Jupyter notebooks and our overall philosophy regarding interactive docs, read more here. (TODO)


# `ipfs-log`

## Intro

The `Log` provided by the `ipfs-log` package is an implementation of a Conflict-Free Replicated Data Type (CRDT) that utilizes the Interplanetary File System (IPFS) as a storage backend. The functionality in this package forms the backbone of [`orbit-db`](https://github.com/orbitdb/orbit-db).

## Nomenclature and Concepts

### Conflict-Free Replicated Data Type (CRDT)

A **Conflict-Free Replicated Data Type** is a type of log that solves the problem of locally storing, and ultimately merging, distrubuted data sets to other distributed data sets<sup>1</sup>. It allows users to perform operations on *local* databases with the intent of merging or joining those data with the data stored on the devices of other peers in the network.

### Lamport Clock

To achieve successful merging - merging that is properly *associative* - entries are timestamped with something called a Lamport Clock<sup>2</sup>. Lamport clocks are instantiation of vector clocks, which means each timestamp is a pair of values: the timestamp of the entry, and an identifier of the user or device that generated the entry.

In the case of `ipfs-log`, the identifier is the public key of the IPFS node where the entries are initially generated.

```javascript
// Lamport Clock Object
{
  id: '042750228c5d81653e5142e6a56d55...e5216b9a6612dbfc56e906bdbf34ea373c92b30d7',
  time: 0
}
```

### Heads and Tails

The **head** of a log is an entry that is not referenced by any other entry

The **tail** of a log points to another entry that is not in the input array (or no entries?)

### G-Set

`ipfs-log` specifically uses a G-Set CRDT, which in practice means "append-only".

```javascript
class GSet {
  constuctor (values) {}
  append (value) {}
  merge (set) {}
  get (value) {}
  has (value) {}
  get values () {}
  get length () {}
}
```

#### References

1. https://citemaster.net/get/10b50274-7bc5-11e5-8aa1-00163e009cc7/p558-lamport.pdf
2. https://hal.inria.fr/inria-00555588


## Table of Contents

1. Front Matter
  1. Intro
  2. Nomenclature
  3. About the Jupyter Notebooks
2. [Prerequisites](#Prerequisites)
  1. [IPFS Node](#IPFS-Node)
  2. [Access Controller](#Access-Controller)
  3. [Identity](#Identity)
3. [Usage](#Usage)
  1. [Creating Logs](#Creating-Logs)
  3. [Manipulating Logs](#Manipulating-Logs)
  4. [Joining Logs](#Joining-Logs)
4. [API Documentation](#API-Documentation)
  1. [Log](#Log)

In [1]:
// const Log = require("ipfs-log")
const Log = require("./src/log") // if running from within the repo

## Prerequisites

For a minimum viable `ipfs-log`, you need three things: a running IPFS node, an access controller, and an identity.

### IPFS Node

For our examples, we'll switch between a node.js [`js-ipfs`](https://github.com/ipfs/js-ipfs) instance, and a go node using [`js-ipfs-api`](https://github.com/ipfs/js-ipfs-api).

In [2]:
const IPFS = require("ipfs")
const IpfsApi = require("ipfs-api")

In [3]:
// Default config
var ipfs = new IPFS()
var ipfsapi = new IpfsApi("localhost", 5001)

^ Always a good sign.

### Access Controller

TODO: Details

In [4]:
const AccessController = require('./src/default-access-controller')

In [5]:
var testACL = new AccessController()

### Identity

TODO: Details


In [6]:
const IdentityProvider = require('orbit-db-identity-provider')
const Keystore = require('orbit-db-keystore')

In [7]:
const keystore = Keystore.create("./test/fixtures/keys")
const identitySignerFn = async (id, data) => {
    const key = await keystore.getKey(id)
    return keystore.sign(key, data)
}

let testIdentityA, testIdentityB;
(async () => { 
    testIdentityA = await IdentityProvider.createIdentity(keystore, 'userA', identitySignerFn)
    testIdentityB = await IdentityProvider.createIdentity(keystore, 'userB', identitySignerFn)
    testIdentityC = await IdentityProvider.createIdentity(keystore, 'userC', identitySignerFn)
})()

## Usage

With the following, we can create a minimum viable log:
* a working IPFS node
* access controller
* an identity

The full signature is 

```javascript
new Log(ipfs, access, identity, [logId], [entries], [heads], [clock])
```

We'll more into the optional params now. For now, let's create a couple "minimum viable" logs.

### Creating Logs

In [8]:
var log = new Log(ipfs, testACL, testIdentityA)   
var log2 = new Log(ipfsapi, testACL, testIdentityB)

If you don't supply a `logId`, the current javascript timestamp will be used.

In [9]:
log.id

'1542128390806'

If no log `entries` are specified, the log's `length` will be 0.

In [10]:
log2.length

0

`ipfs-log` uses Lamport Clocks, which are a type of vector clock. By tracking both the ID and timestamp of each entry, logs can be merged with other logs and still produce a unique, sorted, set of entries to be processed. This data type is compatible with any "pure" function. 

In [11]:
log.clock

LamportClock {
  id:
   '042750228c5d81653e5142e6a56d5551231649160f77b25797dc427f8a5b2afd650acca10182f0dfc519dc6d6e5216b9a6612dbfc56e906bdbf34ea373c92b30d7',
  time: 0 }

In [12]:
log2.clock

LamportClock {
  id:
   '04e9224ee3451772f3ad43068313dc5bdc6d3f2c9a8c3a6ba6f73a472d5f47a96ae6d776de13f2fc2076140fd68ca900df2ca4862b06192adbf8f8cb18a99d69aa',
  time: 0 }

### Manipulating Logs

Now that we have our logs, let's create some entries!

TODO: Entry Signature

In [13]:
const Entry = require("./src/entry")
const Clock = require('./src/lamport-clock')

### Adding entries at log creation


By utilizng the first of our optional params, we can create a log with entries right from the start. Observe how the length and clock values change. Please note that with this method you have to _manually_ set the clock values for each entry.

In [14]:
let log3;

(async() => {
    var entry1 = await Entry.create(ipfs, testIdentityC, 'C', 'entry1', [], new Clock('C', 0))
    var entry2 = await Entry.create(ipfs, testIdentityC, 'C', 'entry2', [], new Clock('C', 1))
    log3 = new Log(ipfs, testACL, testIdentityC, null, [entry1, entry2])
})()

Swarm listening on /ip4/127.0.0.1/tcp/4003/ws/ipfs/QmTnrsWEWwXhAWP1YEKvC3qQUt3MYprXmTmoRmDY4bwwBj
Swarm listening on /ip4/127.0.0.1/tcp/4002/ipfs/QmTnrsWEWwXhAWP1YEKvC3qQUt3MYprXmTmoRmDY4bwwBj
Swarm listening on /ip4/10.0.1.112/tcp/4002/ipfs/QmTnrsWEWwXhAWP1YEKvC3qQUt3MYprXmTmoRmDY4bwwBj
Swarm listening on /ip4/172.17.0.1/tcp/4002/ipfs/QmTnrsWEWwXhAWP1YEKvC3qQUt3MYprXmTmoRmDY4bwwBj


In [15]:
log3.length

2

In [16]:
log3.clock

LamportClock {
  id:
   '045d10320c2a75982d55e7e487db235341ac0a09a36252de6f3e959b9e249841a4bf4cfb909ec8c801ceeb0679586312e1830a753800ee351da54eb20e401df592',
  time: 1 }

### Adding entries after log creation

Note that the clock in the entries will be ignored and are therefor not necessary for this method.

In [17]:
(async () => {
    var entry3 = await Entry.create(ipfs, testIdentityC, 'C', 'entry3', [])
    var entry4 = await Entry.create(ipfs, testIdentityC, 'C', 'entry4', [])
    var entry5 = await Entry.create(ipfs, testIdentityC, 'C', 'entry5', [])
    log3.append(entry3)
    log3.append(entry4)
    log3.append(entry5)
})()

In [18]:
log3.length

2

In [19]:
log3.clock

LamportClock {
  id:
   '045d10320c2a75982d55e7e487db235341ac0a09a36252de6f3e959b9e249841a4bf4cfb909ec8c801ceeb0679586312e1830a753800ee351da54eb20e401df592',
  time: 4 }

### Joining Logs

TODO: Show the magic

## API Documentation


### `class Log`

Creates a new Log instance

In [42]:
var log5 = new Log(ipfs, testACL, testIdentityA)

#### Properties

##### `id`

The ID of the log. If one is not specified it will default to JS microtime.

In [54]:
log5.id

'1542130188333'

##### `values`

Returns an `Array` of [entries](https://github.com/haadcode/ipfs-log/blob/master/src/entry.js) in the log.
 The values are in linearized order according to their [Lamport clocks](https://en.wikipedia.org/wiki/Lamport_timestamps).

In [55]:
log3.values // full array of all values
log3.values.map(v => v.hash)

[ 'Qmb1aXiPJVMTRQnhwa1cFcD8DuSaHoUBjhd4tV3S8ELGSS',
  'QmWMvKh7J37fL2dS287pPYTT3xJPYWwWqTuozh3QxwsdR9',
  'QmUWksAJvrZEMv378Ky7y5dDTXqCwKXCo1CMtDf1mL3suA',
  'QmVPREX2YMpLjYsdMyDbbFAT62kBLKLTCgkzuK2ng5TSrP',
  'QmVtADcWAc8dyBEsg9oVCMPSSSMbXM3oAG5aDj6TyJTuL8',
  'QmWpE1yWphMJAWZrtx5KueJA5aAzGGErFmtqQaBCkhnkPW' ]

##### `length`

Returns the number of entries in the log.

In [48]:
log3.length // Also equivalent to log3.values.length

6

##### `clock`

Returns the current timestamp of the log in Lamport Clock.

In [49]:
log3.clock

LamportClock {
  id:
   '045d10320c2a75982d55e7e487db235341ac0a09a36252de6f3e959b9e249841a4bf4cfb909ec8c801ceeb0679586312e1830a753800ee351da54eb20e401df592',
  time: 5 }

##### `heads`

Returns the heads of the log. Heads are the entries that are not referenced by other entries in the log.

In [50]:
log3.heads // full array of all values
log3.heads.map(h => h.hash)

[ 'QmWpE1yWphMJAWZrtx5KueJA5aAzGGErFmtqQaBCkhnkPW' ]

##### `tails`

Return the tails of the log. Tails are the entries that reference other entries that are not in the log.

In [51]:
log3.tails // full array of all values
log3.tails.map(h => h.hash)

[ 'Qmb1aXiPJVMTRQnhwa1cFcD8DuSaHoUBjhd4tV3S8ELGSS',
  'QmWMvKh7J37fL2dS287pPYTT3xJPYWwWqTuozh3QxwsdR9' ]

#### Methods

##### `get(multihash)`
> @param multihash <br />
> @returns whatever
Find an entry by multihash (TODO: link to multihash docs)

In [27]:
var tail_entry = log3.get("QmWMvKh7J37fL2dS287pPYTT3xJPYWwWqTuozh3QxwsdR9")
tail_entry.payload

'entry2'

##### `has(multihash)`

Verify that a certain entry exists in a log

In [28]:
log3.has("Qmb1aXiPJVMTRQnhwa1cFcD8DuSaHoUBjhd4tV3S8ELGSS")

true

##### `traverse(rootEntry, amount)`

TODO: Need explainer and compelling example

In [29]:
var headHash = log3.heads[0].hash
var headEntry = log3.get(headHash)
log3.traverse([headEntry], 2)

{ QmVtADcWAc8dyBEsg9oVCMPSSSMbXM3oAG5aDj6TyJTuL8: 'QmVtADcWAc8dyBEsg9oVCMPSSSMbXM3oAG5aDj6TyJTuL8',
  Qmb1aXiPJVMTRQnhwa1cFcD8DuSaHoUBjhd4tV3S8ELGSS: 'Qmb1aXiPJVMTRQnhwa1cFcD8DuSaHoUBjhd4tV3S8ELGSS' }

##### `append(entry, [pointerCount = 1])`

Adds a new entry to an existing log

Append an entry to the log. Returns a *Promise* that resolves to the updated `Log`.

`ipfs` IPFS instance.

`log` Log to append to.

`data` can be any type of data: Number, String, Object, etc. It can also be an instance of [Entry](https:/
/github.com/haadcode/ipfs-log/blob/master/src/entry.js).


TODO: pointerCount example

In [30]:
(async () => {
    var entry6 = await Entry.create(ipfs, testIdentityC, 'C', 'entry6', [])
    log3.append(entry6)
    console.log(log3.length)
})()

5


##### `join(log, [size = -1])`

Join the log with another log. Returns a Promise that resolves to a `Log` instance. The size of the joined
 log can be specified by giving `length` argument.

TODO: Size example, also does this even work??

In [31]:
(async () => {
    var entry1B = await Entry.create(ipfs, testIdentityB, 'B', 'entry1B', [])
    var entry2B = await Entry.create(ipfs, testIdentityB, 'B', 'entry2B', [])
    var entry3B = await Entry.create(ipfs, testIdentityB, 'B', 'entry3B', [])
    log2.append(entry1B)
    log2.append(entry2B)
    log2.append(entry3B)
    
    console.log(log3.length, log2.length)
    log3.join(log2)
    console.log(log3.length)
})()

5 0
5


##### `toJSON()`

Outputs a JSON representatiopn of the log

In [32]:
log3.toJSON()

{ id: '1542128390966',
  heads: [ 'QmVtADcWAc8dyBEsg9oVCMPSSSMbXM3oAG5aDj6TyJTuL8' ] }

##### `toSnapshot()`

In [38]:
Object.keys(log3.toSnapshot())

[ 'id', 'heads', 'values' ]

##### `toBuffer()`

Converts the log to a `Buffer` that contains the log as JSON.stringified `string`. Returns a `Buffer`.

In [40]:
log3.toBuffer()
log3.toBuffer().toString()

'{"id":"1542128390966","heads":["QmWpE1yWphMJAWZrtx5KueJA5aAzGGErFmtqQaBCkhnkPW"]}'

##### `toString(payloadMapper)`

Returns the log values as a nicely formatted string.

In [41]:
logString = log2.toString(m => m.hash)

'QmNPrM8mnuBoJyhnhbpZKQpKzY7aCMgUfRjpJAbcmjiYrm\nQmRCKGimSTvkNemDHLYSe3QD1jF2DhRLiPGeJ6Wnort7TL\nQmTdrm59pikFbh4UAC3qkJtyVDqm3y1zV2A2uKTLCiUvSf'

##### (Static) `Log.isLog(log)` 

Check if an object is a `Log` instance.

In [36]:
console.log(Log.isLog(log3))
console.log(Log.isLog("hello"))

true
false


##### `toMultihash()`

Writes the log to IPFS and returns the Multihash of the log. Returns a `Promise` that resolves to a Base58
 encoded `string`.

In [37]:
log3.toMultihash()

'QmSEU7xKrA4Ukj25FG2PhY4LJtUHJvR9nhJvAFFdA4CFhK'

##### `static async Log.fromMultihash(ipfs, access, identity, hash, [length = -1], exclude, onProgressCallback)`

Create a log from multihash
   * @param {IPFS}   ipfs        An IPFS instance
   * @param {string} hash        Multihash (as a Base58 encoded string) to create the log from
   * @param {Number} [length=-1] How many items to include in the log
   * @param {Function(hash, entry, parent, depth)} onProgressCallback
   * @return {Promise<Log>}      New Log

In [ ]:
(async() => {
    var log3_hash = await log3.toMultihash()
    var log4 = await Log.fromMultihash(ipfs, testACL, testIdentityC, log3_hash)
    console.log(log4.length)
})()

##### `static async Log.fromEntryHash (ipfs, access, identity, hash, id, [length = -1], exclude, onProgressCallback)`

TODO: Examples here

##### `static async Log.fromJSON (ipfs, access, identity, json, length = -1, timeout, onProgressCallback)`

##### `static async fromEntry (ipfs, access, identity, sourceEntries, length = -1, exclude, onProgressCallback)`

Create a `Log` from an `Entry`.

Creating a log from an entry will retrieve entries from IPFS, thus causing side effects.

##### ` static findHeads (entries)`

##### `static findTails (entries)`

##### `static findTailHashes (entries)`

##### `  static Log.difference (a, b)`

### LogIO
#### toMultiHash
#### fromMultiHash
#### fromEntryHash
#### fromJSON
#### fromEntry

### LamportClock
#### constructor
#### tick
#### merge
#### clone
#### compare

### Entry
#### create
#### verify
#### toBuffer
#### toMultiHash
#### fromMultiHash
#### isEntry
#### compare
#### isEqual
#### isParent
#### findChildren

### EntryIO
#### fetchParallel
#### fetchAll

### Utility Functions
#### (src/log-sorting.js) LastWriteWins
#### (src/log-sorting.js) SortByClocks
#### (src/log-sorting.js) SortByClockId
#### (src/utils/difference.js) difference
#### (src/utils/find-uniques.js) findUniques
#### (src/utils/intersection.js) intersection
#### (src/utils/is-defined.js) is-defined

### Errors
#### IPFSNotDefinedError
#### LogNotDefinedError
#### NotALogError